In [ ]:
# install and import pygsheets
!pip install pygsheets
import pygsheets

#  Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# connect google sheets
import gspread
# import pygsheets
gc = gspread.authorize(GoogleCredentials.get_application_default())


import json
from google.oauth2 import service_account

with open('service_account.json') as source:
   info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

client = pygsheets.authorize(service_account_file='service_account.json')


# import pandas
import pandas as pd
from functools import reduce


In [ ]:
weekFolderID = '1EjFK6sBqh8hB6-A45hIStW2P6hi9sJcW'

Combine Performance File

In [ ]:
# Append all files in folder into a listframe

listframe = []

weekFolder = drive.ListFile({'q': "'"+weekFolderID+"' in parents and  mimeType = 'application/vnd.google-apps.folder'"}).GetList()
day = 'Jul 09'

for folder in weekFolder:
  if folder['title'] == 'Performance':
    # https://developers.google.com/drive/v2/web/search-parameters
    files = drive.ListFile({'q': "title contains '"+day+"'and not title contains \
                            '(7)' and not title contains '(6)' and '"+folder['id']+"' in parents \
                            and mimeType = 'text/csv' and trashed = false "}).GetList()
    for file in files:
      # file = drive.CreateFile({'id':file['id']})
      # file['title'] = file['title'].replace("_Jul_"," Jul ") # Change title.
      # file.Upload()
      print(file['title'])
      # download
      fileDownloaded = drive.CreateFile({'id':file['id']})

      # load
      fileDownloaded.GetContentFile('performance.csv')

      # read
      df = pd.read_csv('performance.csv', delimiter=',' )

      # append
      listframe.append(df)

# combine all files in the list into a dataframe
wkperffile = pd.concat(listframe)

# convert hour online to just hours
wkperffile['Hours online'] = pd.to_timedelta(wkperffile['Hours online']).dt.total_seconds()/3600

# convert strings to numerics
wkperffile[['Name', 'Net Fares', 'Per trip', 'Per hour online', \
            'Per km on trip']] = pd.to_numeric(wkperffile[['Name', \
                                'Net Fares', 'Per trip', 'Per hour online', 'Per km on trip']], errors='coerce')

distancepertrip =wkperffile['Distance per trip'].str.split(" ",n=0,expand=True)
wkperffile['Distance per trip'] = distancepertrip[0].fillna(0)
wkperffile['Distance per trip'] = pd.to_numeric(wkperffile['Distance per trip'])

acceptancerate =wkperffile['Acceptance rate'].str.split("%",n=0,expand=True)
wkperffile['Acceptance rate'] = acceptancerate[0].fillna(0)
wkperffile['Acceptance rate'] = pd.to_numeric(wkperffile['Acceptance rate'],errors='coerce')/100

cancellationrate =wkperffile['Driver cancellation rate'].str.split("%",n=0,expand=True)
wkperffile['Driver cancellation rate'] = cancellationrate[0].fillna(0)
wkperffile['Driver cancellation rate'] = pd.to_numeric(wkperffile['Driver cancellation rate'],errors='coerce')/100

wkperffile['Ratings'] = pd.to_numeric(wkperffile['Ratings'],errors='coerce')
wkperffile['Lifetime rating'] = pd.to_numeric(wkperffile['Lifetime rating'],errors='coerce')


# aggregate metrics by names
# wkperffile['Name'] = pd.Series(wkperffile['Name']).str.title()
# wkperffile = wkperffile.groupby('Name').agg({'Net Fares':'sum', 'Per trip':'mean','Per hour online':'mean', 'Per km on trip':'mean',\
#                                              'Trips':'sum', 'Hours online':'sum', 'Trips per hour':'mean', 'Distance per trip':'mean',\
#                                               'Ratings':'mean','Lifetime rating':'mean','Acceptance rate':'mean','Driver cancellation rate':'mean'}).reset_index()
pd.set_option('display.max_columns', 500)
print(wkperffile.columns)

Combine Payments file

In [ ]:
#  Append all files in folder into a listframe
paymentslistframe = []

for folder in weekFolder:
  if folder['title'] == 'Payments':
    files = drive.ListFile({'q': "title contains '"+day+"'and not title contains \
                            '(7)' and not title contains '(6)' and '"+folder['id']+"' in parents \
                            and mimeType = 'text/csv' and trashed = false "}).GetList()
    for file in files:
      # print(file['title'])

      # download
      fileDownloaded = drive.CreateFile({'id':file['id']})

      # load
      fileDownloaded.GetContentFile('payments.csv')

      # read
      df = pd.read_csv('payments.csv', delimiter=',' )

      # append
      paymentslistframe.append(df)

# combine all files in the list into a dataframe
combinedPayments = pd.concat(paymentslistframe)

# concatenate the names
combinedPayments['Name'] = combinedPayments['firstName'] + " " + combinedPayments['lastName']

# pick only fullname, amount and itemtype
paymentsfile = combinedPayments[['driverUUID','Name','amount','itemType']]

# get total cash collected
# get cash collected itemtype
cashCollected = paymentsfile.loc[paymentsfile['itemType'] == 'cash_collected']

# sum and count amounts by names to get the cash collected per driver
cashCollected['Name'] = pd.Series(cashCollected['Name']).str.title()
cashCollected = cashCollected.groupby('Name').agg({'driverUUID':'first','amount':['sum','count']}).reset_index()
cashCollected.columns = ['Name','driverUUID', 'cashCollected','cashTrips']

print(cashCollected.head(400))

Combine All Drivrs File

In [ ]:
#  Append all files in folder into a listframe
alldriverslistframe = []

for folder in weekFolder:
  if folder['title'] == 'All Drivers':
    files = drive.ListFile({'q': "title contains '"+day+"'and not title contains '(7)' and not title contains '(6)' and '"+folder['id']+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
    for file in files:
      # print(file['title'])

      # download
      fileDownloaded = drive.CreateFile({'id':file['id']})

      # load
      fileDownloaded.GetContentFile('alldrivers.csv')

      # read
      df = pd.read_csv('alldrivers.csv', delimiter=',' )

      # append
      alldriverslistframe.append(df)

# combine all files in the list into a dataframe
combinedAlldrivers = pd.concat(alldriverslistframe)

print(combinedAlldrivers.columns)
# concatenate the names
# combinedAlldrivers['Name'] = combinedAlldrivers['First Name'] + " " + combinedAlldrivers['Last Name']

# pick only fullname, uber fees
alldriversfile = combinedAlldrivers[['Driver ID','Uber Fee']]

# sum uber fees by drivers
alldriversfile['Name'] = pd.Series(alldriversfile['Driver ID']).str.title()
alldriversfile = alldriversfile.groupby('Driver ID').sum().reset_index()
alldriversfile.columns = ['driverUUID', 'Uber Fee']

print(alldriversfile.head(400))

Get Moove Details

In [ ]:
# get moove details
mooveDetailList =[]
worksheet = gc.open_by_key('1Q3BJcXwRncZiiB3GAYw1CNN-RWOWjt4AyUHxjs_Kztg').sheet1
for i in range(2,5):
  columns = worksheet.col_values(i)
  mooveDetailList.append(columns[1:])

# print(mooveDetailList)

# convert list to data frame
mooveDetails = pd.DataFrame(mooveDetailList)
mooveDetails = mooveDetails.T
mooveDetails.columns = ['Name','Product','DRN']
mooveDetails['Name'] = pd.Series(mooveDetails['Name']).str.title()
print(mooveDetails)

Virtual Accounts Data

In [ ]:
fileid = '1DRk8HYz_HPu_K8jzGEQB_N2hu5StgaWo'

folder = drive.ListFile({'q': "'"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
for file in folder:
  # print(file['title'])

  # download
  fileDownloaded = drive.CreateFile({'id':file['id']})

  # load
  fileDownloaded.GetContentFile('VirtualAccount.csv')

  # read
  VirtualAccountDF = pd.read_csv('VirtualAccount.csv', delimiter=',' )

# select only drn and amount
VirtualAcctDF = VirtualAccountDF[['drn','amount']]

# Sum and VA Amount
VA = VirtualAcctDF.groupby('drn').agg({'amount':['sum','count']}).reset_index()
VA.columns = ['DRN','VAAmount', 'VATrips']

print(VA)

Combine Performance, Earnings and Moove Details

In [ ]:
# join elements of the final payments file
dfs = [alldriversfile, cashCollected]

# join on driverUUID
reconfile = reduce(lambda left,right: pd.merge(left,right,how='left', on='driverUUID'), dfs)

# join on Name
dfs = [wkperffile, reconfile, mooveDetails]
reconfile = reduce(lambda left,right: pd.merge(left,right,how='left', on='Name'), dfs)

# join on DRN
dfs = [reconfile, VA]
reconfile = reduce(lambda left,right: pd.merge(left,right,how='left', on='DRN'), dfs)

reconfile = reconfile.fillna(0)

reconfile['Date'] = day
reconfile['cashCollected'] = -reconfile['cashCollected']
reconfile['grossFares'] = reconfile['Net Fares'] - reconfile['Uber Fee']
reconfile['grossFT'] = reconfile['grossFares']/reconfile['Trips']
reconfile['Distance on Trip'] = reconfile['Distance per trip']*reconfile['Trips']
reconfile['tripsAccepted'] = round(reconfile['Trips']/(1-reconfile['Driver cancellation rate']),0)

reconfile = reconfile[['Date','Name', 'driverUUID', 'DRN', 'Product', 'grossFares',
       'grossFT', 'Net Fares', 'Per trip', 'Per hour online', 'Per km on trip',
       'Trips', 'tripsAccepted', 'Hours online', 'Trips per hour', 'Distance per trip', 'Distance on Trip',
       'Ratings', 'Lifetime rating', 'Acceptance rate',
       'Driver cancellation rate', 'Uber Fee', 'cashCollected',
       'cashTrips', 'VAAmount', 'VATrips']]
reconfile.drop_duplicates(subset=['Name', 'driverUUID', 'DRN','Trips'],inplace=True)
print(reconfile)

Write reconciled files into google sheets

In [ ]:
reconSheet = client.open_by_key('1Q3BJcXwRncZiiB3GAYw1CNN-RWOWjt4AyUHxjs_Kztg') 
wksalldriversfile = reconSheet.worksheet_by_title("dailyData")
lastrow = len(wksalldriversfile.get_values('A1','A10000'))+1

wksalldriversfile.set_dataframe(reconfile,(lastrow,1),copy_head=False,nan='')